Upscaling 25x to 40x images of NADH type only.
Importing CARE model from another module.

Last updated: 06.29

Change: Testing premade CARE package




In [ ]:
# !pip install tifffile
# !pip install sklearn
# !pip install scikit-image
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing dependencies
import matplotlib.pyplot as plt
import sklearn
import sklearn.model_selection
import skimage
import math
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import math
import keras.backend as K
from datetime import datetime
import fractions
import itertools
import tqdm
from keras.utils.conv_utils import normalize_tuple

In [ ]:
from google.colab import drive
import sys
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# DATA_PATH = '/cluster/tufts/georgakoudi_lab01/tdinh02/npz/Cervix_all_data_original.npz'
# LABEL_PATH = '/cluster/tufts/georgakoudi_lab01/tdinh02/npz/Cervix_all_data_labels_original_3D.npz'\
# MAIN_PATH = '/cluster/tufts/georgakoudi_lab01/tdinh02/objective_transform/'

DATA_PATH = '/npz/Cervix_all_data_original.npz'
LABEL_PATH = '/npz/Cervix_all_data_labels_original.npz'
# MAIN_PATH= r"/content/drive/MyDrive/Research and Grad/Diamond Program/objective_transfer/deep_learning" # Artem's Drive
MAIN_PATH = "/content/drive/MyDrive" # Filip's drive
now = datetime.now() # current date and time
CURR_DATE = now.strftime("%m-%d-%Y")

def config(patch_size, depth):
  return {
        'img_size': 256,
        'learning_rate': 1e-5,
        'batch_size': 16,
        'alpha': 0.84,
        'patch_size':patch_size,
        'input_shape': [patch_size, patch_size],
        'kern_size':3,
        'n_depth': depth,
        'first_depth': 32,
        'dropout': 0,
        'epoch':1,
        'lr_decay_factor':0.97,
        'lr_decay_patience':10,
  }

MODEL_ROOT_NAME = "CARE_patch_depth_tune_0726"

In [ ]:
# importing yaml config file and resetting working direcgory
os.chdir(MAIN_PATH+'/CAREstd/')
import import_ipynb
from CARE_util import *

importing Jupyter notebook from CARE_util.ipynb


In [ ]:
# def ssim(y_true, y_pred):
#     return (tf.image.ssim(y_true, y_pred,1,k2=0.05)) # sliding Gaussian window as mentioned in Wikipedia
# smaller filter size, should avoid blurring
def ssim(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, 1, filter_size=3, filter_sigma=0.5, k2=0.05)

w =  (0.0448, 0.2856, 0.3001)
def mssim(y_true, y_pred):
   return tf.image.ssim_multiscale(y_true, y_pred, 1, filter_size=11,power_factors=w, filter_sigma=1.5, k2=0.05)

def psnr(y_true, y_pred):
    '''
    Computs the peak signal-to-noise ratio between two images. Note that the
    maximum signal value is assumed to be 1.
    '''
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

def SSIM_loss(y_true, y_pred):
    return 1-((ssim(y_true, y_pred)+1)*0.5) # chanfe to 2

# Use cw_ssim
#
# from scipy import signal
# def cw_ssim(y_true, y_pred, width=30):
#         """Compute the complex wavelet SSIM (CW-SSIM) value from the reference
#         image to the target image.
#         Args:
#           target (str or PIL.Image): Input image to compare the reference image
#           to. This may be a PIL Image object or, to save time, an SSIMImage
#           object (e.g. the img member of another SSIM object).
#           width: width for the wavelet convolution (default: 30)
#         Returns:
#           Computed CW-SSIM float value.
#         """
#         k = 0.01 #k (float): CW-SSIM configuration variable (default 0.01)
#         # Define a width for the wavelet convolution
#         widths = np.arange(1, width+1)

#         # Use the image data as arrays
#         sig1 = y_pred.numpy().flatten()
#         sig2 = y_true.numpy().flatten()

#         # Convolution
#         cwtmatr1 = signal.cwt(sig1, signal.ricker, widths)
#         cwtmatr2 = signal.cwt(sig2, signal.ricker, widths)

#         # Compute the first term
#         c1c2 = np.multiply(abs(cwtmatr1), abs(cwtmatr2))
#         c1_2 = np.square(abs(cwtmatr1))
#         c2_2 = np.square(abs(cwtmatr2))
#         num_ssim_1 = 2 * np.sum(c1c2, axis=0) + k
#         den_ssim_1 = np.sum(c1_2, axis=0) + np.sum(c2_2, axis=0) + k

#         # Compute the second term
#         c1c2_conj = np.multiply(cwtmatr1, np.conjugate(cwtmatr2))
#         num_ssim_2 = 2 * np.abs(np.sum(c1c2_conj, axis=0)) + k
#         den_ssim_2 = 2 * np.sum(np.abs(c1c2_conj), axis=0) + k

#         # Construct the result
#         ssim_map = (num_ssim_1 / den_ssim_1) * (num_ssim_2 / den_ssim_2)

#         # Average the per pixel results
#         index = np.average(ssim_map)
#         return index

def SSIML1_loss(y_true, y_pred, alpha=0.84):
  # alpha = 0.84
  ssim_partial = 1-((ssim(y_true, y_pred)+1)*0.5)
  mae_partial = tf.keras.losses.mae(
        *[tf.keras.backend.batch_flatten(y) for y in [y_true, y_pred]])
  return alpha*ssim_partial  + (1-alpha)*mae_partial

def mov_var(image):
  dtype = tf.float32
  img_height = tf.shape(image)[1]
  img_width = tf.shape(image)[2]
  mean_filter = tf.ones((3,3),dtype) / 9
  img_mean = tf.nn.conv2d(image[:,:,:,:],
                          mean_filter[:,:,tf.newaxis,tf.newaxis],
                          [1,1,1,1],'VALID')
  img_clip = image[:, 1:-1, 1:-1,:]
  # Difference between pixel intensity and its block mean
  x_diff = tf.math.squared_difference(img_clip, img_mean) / 8
  return x_diff

def genSSIML1_loss(alpha=0.84):
  def SSIM_L1_loss(y_true, y_pred):
    ssim_partial = 1-((cw_ssim(y_true, y_pred)+1)*0.5)
    mae_partial = tf.keras.losses.mae(
          *[tf.keras.backend.batch_flatten(y) for y in [y_true, y_pred]])
    return alpha*ssim_partial  + (1-alpha)*mae_partial
  return SSIM_L1_loss

def genSSIMVar_loss(alpha=0.84):
  def SSIMVar_loss(y_true, y_pred):
      SSIM = 1-((ssim(y_true, y_pred)+1)*0.5)
      MAE = tf.keras.losses.mae(
          *[tf.keras.backend.batch_flatten(y) for y in [mov_var(y_true), mov_var(y_pred)]])
      return alpha * SSIM + (1-alpha) * MAE
  return SSIMVar_loss

def genSSIMVarL1_loss(alpha=0.84):
  def SSIMVarL1_loss(y_true, y_pred):
      SSIM = 1-((ssim(y_true, y_pred)+1)*0.5)
      MAE = tf.keras.losses.mae(
          *[tf.keras.backend.batch_flatten(y) for y in [mov_var(y_true), mov_var(y_pred)]])
      MAE2 = tf.keras.losses.mae(
          *[tf.keras.backend.batch_flatten(y) for y in [y_true, y_pred]])
      return alpha * SSIM + ((1-alpha)/4) * MAE + (3*(1-alpha)/4) * MAE2
  return SSIMVarL1_loss

In [ ]:
# HELPER FUNCTIONS

def create_patches(img, patch_shape, slide):
    # returns stack of patches and number of patches
    patch_img = skimage.util.view_as_windows(img, (patch_shape,patch_shape), step=patch_shape-slide)
    patch = patch_img.reshape(patch_img.shape[0]*patch_img.shape[1],patch_shape,patch_shape) # more time efficient
    return patch

def patchify(input, patch_shape, slide):
    # getting number of input images
    len_to_allocate = int(np.shape(input)[0]*((slide-np.shape(input)[1]) / (slide-patch_shape))**2)
    data = np.zeros((len_to_allocate,patch_shape,patch_shape))
    count = 0
    for i in range(np.shape(input)[0]):
      A = create_patches(input[i], patch_shape, slide)
      # print("[:,:,:]A", np.amax(A[:,:,:]))
      data[count:count+len(A),:,:] = A[:,:,:]
      # print("data[count:count+len(A),:,:]", np.amax(data[count:count+len(A),:,:]))
      count = count + len(A)
    print("      [PATCHIFYING COMPLETED] output shape, slide: ",np.shape(data),slide,"; number of images: ", np.shape(input)[0], ", number of patches: ", np.shape(data)[0])
    return data


def load_data(path, expand=False, patch=None):
    # Loading preprocessed image patches and adding 4th arbitrary dimension
    b = np.load(path)
    training_data = b['t']
    val_data = b['v']

    # not supporting 3D stacks at the moment
    if len(np.shape(training_data)) > 3:
      print("3D STACKS ARE NOT SUPPORTED")
      return []
    # patchify if patch size is passed
    if patch != None:
      slide = int(patch/2)
      # slide = 0
      training_data = patchify(training_data, patch, slide)
      val_data = patchify(val_data, patch, slide)

    # if len(np.shape(training_data))>3:
    #   # for 3D only, changing dimensions
    #   training_data = np.transpose(training_data, (0, 3, 1, 2,4))
    #   val_data = np.transpose(val_data, (0, 3, 1, 2,4))
    if expand == True:
      training_data = expandLastDim(training_data)
      val_data = expandLastDim(val_data)
    res = [training_data, val_data]
    return res

def expandLastDim(data):
    return np.expand_dims(data, -1)

# add arbitrary channel dimension and augments data on flow, also splits data into batches
class DataGenerator:
    '''
    Generates batches of image pairs with real-time data augmentation.
    Parameters
    ----------
    shape: tuple of int
        Shape of batch images (excluding the channel dimension).
    batch_size: int
        Batch size.
    transform_function: str or callable or None
        Function used for data augmentation. Typically you will set
        ``transform_function='rotate_and_flip'`` to apply combination of
        randomly selected image rotation and flipping.  Alternatively, you can
        specify an arbitrary transformation function which takes two input
        images (source and target) and returns transformed images. If
        ``transform_function=None``, no augmentation will be performed.
    intensity_threshold: float
        If ``intensity_threshold > 0``, pixels whose intensities are greater
        than this threshold will be considered as foreground.
    area_ratio_threshold: float between 0 and 1
        If ``intensity_threshold > 0``, the generator calculates the ratio of
        foreground pixels in a target patch, and rejects the patch if the ratio
        is smaller than this threshold.
    scale_factor: int != 0
        Scale factor for the target patch size. Positive and negative values
        mean up- and down-scaling respectively.
    '''
    def __init__(self,
                 shape,
                 batch_size,
                 transform_function='rotate_and_flip',
                 intensity_threshold=0.0,
                 area_ratio_threshold=0.0,
                 scale_factor=1):
        def rotate_and_flip(x, y, dim):
            if dim == 2:
                k = np.random.randint(0, 4)
                x, y = [np.rot90(v, k=k) for v in (x, y)]
                if np.random.random() < 0.5:
                    x, y = [np.fliplr(v) for v in (x, y)]
                return x, y
            elif dim == 3:
                k = np.random.randint(0, 4)
                x, y = [np.rot90(v, k=k, axes=(1, 2)) for v in (x, y)]
                if np.random.random() < 0.5:
                    x, y = [np.flip(v, axis=1) for v in (x, y)]
                if np.random.random() < 0.5:
                    x, y = [np.flip(v, axis=0) for v in (x, y)]
                return x, y
            else:
                raise ValueError('Unsupported dimension')

        self._shape = tuple(shape)
        self._batch_size = batch_size

        dim = len(self._shape)

        if transform_function == 'rotate_and_flip':
            if shape[-2] != shape[-1]:
                raise ValueError(
                    'Patch shape must be square when using `rotate_and_flip`; '
                    f'Received shape: {shape}')
            self._transform_function = lambda x, y: rotate_and_flip(x, y, dim)
        elif callable(transform_function):
            self._transform_function = transform_function
        elif transform_function is None:
            self._transform_function = lambda x, y: (x, y)
        else:
            raise ValueError('Invalid transform function')

        self._intensity_threshold = intensity_threshold

        if not 0 <= area_ratio_threshold <= 1:
            raise ValueError('"area_ratio_threshold" must be between 0 and 1')
        self._area_threshold = area_ratio_threshold * np.prod(shape)

        self._scale_factor = normalize_tuple(scale_factor, dim, 'scale_factor')
        if any(not isinstance(f, int) or f == 0 for f in self._scale_factor):
            raise ValueError('"scale_factor" must be nonzero integer')

    class _Sequence(tf.keras.utils.Sequence):
        def _scale(self, shape):
            return tuple(
                s * f if f > 0 else s // -f
                for s, f in zip(shape, self._scale_factor))

        def __init__(self,
                     x,
                     y,
                     batch_size,
                     shape,
                     transform_function,
                     intensity_threshold,
                     area_threshold,
                     scale_factor):
            self._batch_size = batch_size
            self._transform_function = transform_function
            self._intensity_threshold = intensity_threshold
            self._area_threshold = area_threshold
            self._scale_factor = scale_factor

            for s, f, in zip(shape, self._scale_factor):
                if f < 0 and s % -f != 0:
                    raise ValueError(
                        'When downsampling, all elements in `shape` must be '
                        'divisible by the scale factor; '
                        f'Received shape: {shape}, '
                        f'scale factor: {self._scale_factor}')

            self._x, self._y = [
                list(m) if isinstance(m, (list, tuple)) else [m]
                for m in [x, y]]
            self._x = np.moveaxis(self._x,0,-1)
            self._y = np.moveaxis(self._y,0,-1)
            if len(self._x) != len(self._y):
                raise ValueError(
                    'Different number of images are given: '
                    f'{len(self._x)} vs. {len(self._y)}')

            if len({m.dtype for m in self._x}) != 1:
                raise ValueError('All source images must be the same type')
            if len({m.dtype for m in self._y}) != 1:
                raise ValueError('All target images must be the same type')
            print(len(self._x))
            for i in range(len(self._x)):
                if len(self._x[i].shape) == len(shape):
                    self._x[i] = self._x[i][..., np.newaxis]

                if len(self._y[i].shape) == len(shape):
                    self._y[i] = self._y[i][..., np.newaxis]

                if len(self._x[i].shape) != len(shape) + 1:
                    raise ValueError(f'Source image must be {len(shape)}D')

                if len(self._y[i].shape) != len(shape) + 1:
                    raise ValueError(f'Target image must be {len(shape)}D')
                if self._x[i].shape[:-1] < shape:
                    raise ValueError(
                        'Source image must be larger than the patch size')

                expected_y_image_size = self._scale(self._x[i].shape[:-1])
                if self._y[i].shape[:-1] != expected_y_image_size:
                    raise ValueError('Invalid target image size: '
                                     f'expected {expected_y_image_size}, '
                                     f'but received {self._y[i].shape[:-1]}')

            if len({m.shape[-1] for m in self._x}) != 1:
                raise ValueError(
                    'All source images must have the same number of channels')
            if len({m.shape[-1] for m in self._y}) != 1:
                raise ValueError(
                    'All target images must have the same number of channels')
            self._batch_x = np.zeros(
                (batch_size, *shape, self._x[0].shape[-1]),
                dtype=self._x[0].dtype)
            self._batch_y = np.zeros(
                (batch_size, *self._scale(shape),self._y[0].shape[-1]),
                dtype=self._y[0].dtype)

        def __len__(self):
            return len(self._x) // self._batch_size # return a dummy value

        def __next__(self):
            return self.__getitem__(0)

        def __getitem__(self, _):
            for i in range(self._batch_size):
                for _ in range(139):
                    j = np.random.randint(0, len(self._x))

                    tl = [np.random.randint(0, a - b + 1)
                          for a, b in zip(
                              self._x[j].shape, self._batch_x.shape[1:])]

                    x = np.copy(self._x[j][tuple(
                        [slice(a, a + b) for a, b in zip(
                            tl, self._batch_x.shape[1:])])])
                    y = np.copy(self._y[j][tuple(
                        [slice(a, a + b) for a, b in zip(
                            self._scale(tl), self._batch_y.shape[1:])])])

                    if (self._intensity_threshold <= 0.0 or
                            np.count_nonzero(y > self._intensity_threshold)
                            >= self._area_threshold):
                        break
                else:
                    import warnings
                    warnings.warn(
                        'Failed to sample a valid patch',
                        RuntimeWarning,
                        stacklevel=3)


                self._batch_x[i], self._batch_y[i] = \
                    self._transform_function(x, y)
            return self._batch_x, self._batch_y

    def flow(self, x, y):
        '''
        Returns a `keras.utils.Sequence` object which generates batches
        infinitely. It can be used as an input generator for
        `keras.models.Model.fit_generator()`.
        Parameters
        ----------
        x: array_like or list of array_like
            Source image(s).
        y: array_like or list of array_like
            Target image(s).
        Returns
        -------
        keras.utils.Sequence
            `keras.utils.Sequence` object which generates tuples of source and
            target image patches.
        '''
        return self._Sequence(x,
                              y,
                              self._batch_size,
                              self._shape,
                              self._transform_function,
                              self._intensity_threshold,
                              self._area_threshold,
                              self._scale_factor)

# CALLBACKS
# tensorboard callback
tb_callback = tf.keras.callbacks.TensorBoard('./logs', update_freq=1,write_images=True,write_steps_per_second=True,)


def build_model(n_dim=2, n_depth=2, kern_size=5, dropout= 0, n_first=32, n_channel_out=1, last_activation='relu', batch_norm=False,shape=(None,None,1)):
    model = common_unet(n_dim,n_depth,n_first=n_first,kern_size=kern_size, dropout=dropout,last_activation=last_activation,batch_norm=batch_norm)(shape)
    model.summary()
    model.save_weights('model.h5')
    return model

# Compile model for training
def compile_model(model, lr, loss=SSIML1_loss):
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
                loss=loss,
                # loss=tf.keras.losses.MeanAbsoluteError(),
                metrics = [{'psnr': psnr, 'ssim': ssim}])
                # metrics = [{'mae': tf.keras.losses.mae}])
    return model

# Create a callback that saves the model's weights every 5 epochs
def save_weight_callback(checkpoint_path, batch_size, epoch_freq=10):
  return tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=epoch_freq*batch_size)

def display_training_stats(history, savepath, stat, axis_name):
    plt.figure(figsize=(10,10))
    plt.plot(history.history[stat], label=stat)
    plt.plot(history.history['val_'+stat], label = 'val_'+stat)
    plt.xlabel(axis_name)
    plt.ylabel(stat)
    plt.legend(loc='upper right')
    plt.savefig(savepath+'_'+stat+'.png')

### **Manual Tuning**

In [ ]:
# for each patch_size, we will train different depths until smalles image shape is 4x4 px
# max depth is determined by log2(patch_size) - 2. ex: log2(64) - 2 = 4 depths, or log2(256) -2 = 6 depths
PATCH_SIZES = [32,64,128,256]

if False:
  res = np.zeros((1,4))
  for patch_size in PATCH_SIZES:
    max_depth = int(math.log2(patch_size)-2)
    depths = np.arange(2,max_depth+1)
    print("MANUAL TUNING PATCHES - patch size: "+ str(patch_size) + ", max_depth: "+str(max_depth)+", all depths: ",depths)
    # for every depth
    for depth in depths:
      print("    [CONFIGURING] "+"patch size: "+ str(patch_size) + ", current depth: "+str(depth))
      config_train = config(patch_size, depth)

      # learning rate reducer callback
      reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=config_train['lr_decay_factor'],min_delta=0,patience=config_train['lr_decay_patience'],)

      # generating data
      data_gen = DataGenerator(
          config_train['input_shape'],
          config_train['batch_size'],
          transform_function=None)
      [training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=patch_size)
      [training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=patch_size)
      tdata = data_gen.flow(*list(zip([training_data,training_data_labels])))
      vdata = data_gen.flow(*list(zip([val_data,val_data_labels])))
      # TRAINING
      model = build_model(kern_size=config_train['kern_size'], n_depth=config_train['n_depth'],shape=(config_train['input_shape'][0],config_train['input_shape'][0],1))
      model = compile_model(model,config_train['learning_rate'])
      print("    [TRAINING] "+"patch size: "+ str(patch_size) + ", current depth: "+str(depth))
      history = model.fit(
                          x=tdata,
                          epochs = config_train['epoch'],
                          validation_data = vdata
                          , callbacks=[reduce_lr,tb_callback])
      print("    [TRAINING FINISHED] "+"p: "+ str(patch_size) + ", d: "+str(depth)+ ", results: val_ssim = ",np.amax(history.history['val_ssim'])," ssim = ",np.amax(history.history['ssim']))
      save_path = MAIN_PATH+"/"+MODEL_ROOT_NAME+"/"+"p_d_valssim_"+str(patch_size)+"_"+str(depth)+"_"+str(np.amax(history.history['val_ssim']))
      print("    [SAVING MODEL] path is: ",save_path)
      model.save(save_path)
      display_training_stats(history,save_path, 'loss', 'Epoch')
      display_training_stats(history,save_path, 'ssim', 'Epoch')
      display_training_stats(history,save_path, 'psnr', 'Epoch')
      res = np.concatenate((res, [[str(patch_size),str(depth),np.amax(history.history['ssim']),np.amax(history.history['val_ssim'])]]), axis=0)
      del training_data, val_data,tdata,vdata, training_data_labels, val_data_labels, model

  print("[TUNING  FINISHED, PRINTING RESULTS]")
  for i in res:
    print(i)

In [ ]:
def _get_gaussian_kernel(dim, size, sigma):
    import tensorflow_probability as tfp
    k = size // 2
    normal = tfp.distributions.Normal(0.0, sigma)
    p = normal.prob(tf.range(-k, size - k, dtype=tf.float32))

    indices = [chr(i) for i in range(105, 105 + dim)]
    eq = ','.join(indices) + '->' + ''.join(indices)
    kernel = tf.einsum(eq, *([p] * dim))
    kernel /= tf.reduce_sum(kernel)
    kernel = kernel[..., tf.newaxis, tf.newaxis]

    return kernel

# RCAN ssim
import keras.backend as K
def ssim_rcan(y_true, y_pred):
    '''
    Computes the structural similarity index between two images. Note that the
    maximum signal value is assumed to be 1.
    References
    ----------
    Image Quality Assessment: From Error Visibility to Structural Similarity
    https://doi.org/10.1109/TIP.2003.819861
    '''

    c1 = 0.01 ** 2
    c2 = 0.03 ** 2

    dim = K.ndim(y_pred) - 2
    if dim not in (2, 3):
        raise NotImplementedError(f'{dim}D SSIM is not suported')

    num_channels = K.int_shape(y_pred)[-1]

    kernel = _get_gaussian_kernel(dim, 11, 1.5)
    conv = K.conv2d if dim == 2 else K.conv3d

    def average(x):
        # channel-wise weighted average using the Gaussian kernel
        return tf.concat(
            [conv(y, kernel) for y in tf.split(x, num_channels, axis=-1)],
            axis=-1)

    ux = average(y_true)
    uy = average(y_pred)

    a = ux * uy
    b = K.square(ux) + K.square(uy)
    c = average(y_true * y_pred)
    d = average(K.square(y_true) + K.square(y_pred))

    lum = (2 * a + c1) / (b + c1)
    cs = (2 * (c - a) + c2) / (d - b + c2)

    return K.mean(K.batch_flatten(lum * cs), axis=-1)

def revert_img(img,original_size, patch_shape, slide):
  # reverts original image and removes overlaps by splitting overlap over 2 images
  step = int(patch_shape-slide)
  reconstructed_arr = np.zeros((original_size,original_size))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      start_x = int(slide/2)
      start_y = int(slide/2)
      end_x = 0
      end_y = 0
      if x == 0:
        start_x = 0
        end_x = int(slide/2)
      if y == 0:
        start_y = 0
        end_y = int(slide/2)
      if x == img.shape[0]-1: end_x = int(slide/2)
      if y == img.shape[1]-1: end_y = int(slide/2)
      x_pos, y_pos = x * step + start_x, y * step + start_y
      reconstructed_arr[x_pos : x_pos + step + end_x, y_pos : y_pos + step + end_y] = img[x, y, start_x:start_x+step+end_x, start_y:start_y+step+end_y]
  return reconstructed_arr

def merge_patches(img, original_size, patch_shape, slide):
  #  merging patches, img is a 3D array of stacked patches
  row_len = int(math.sqrt(img.shape[0]))
  patches = np.zeros((row_len,row_len,patch_shape,patch_shape))
  for r in range(row_len):
      patches[r,:,:,:] = img[r*row_len:r*row_len+row_len,:,:]
  return revert_img(patches,original_size,patch_shape, slide)

def normalize0to1(img):
  # Normalizing images between 0 and 1 and preserving distribution
  img_norm = (img - np.amin(img))/( np.amax(img)- np.amin(img))
  return img_norm

def mergeAndPredict(model,raw,gt,start, full_size, patch_size, overlap, merge=True, normalize=True, clip=False):
  end = int((full_size - overlap) / (patch_size - overlap))**2
  # outputs a 3D list of 3 images in order raw, predicted, ground truth
  pred = model.predict(raw[start:end])

  if len(np.shape(raw)) > 3:
    if merge:
      result = [merge_patches(raw[start:end,:,:,0],full_size,patch_size,overlap), merge_patches(pred[start:end,:,:,0],full_size,patch_size,overlap), merge_patches(gt[start:end,:,:,0],full_size,patch_size,overlap)]
    else: result = [raw[start:end,:,:,0], pred[start:end,:,:,0],gt[start:end,:,:,0]]
  else:
    if merge:
      result = [merge_patches(raw[start:end,:,:],full_size,patch_size,overlap), merge_patches(pred[start:end,:,:,0],full_size,patch_size,overlap), merge_patches(gt[start:end,:,:],full_size,patch_size,overlap)]
    else: result = [raw[start:end,:,:], pred[start:end,:,:,0], gt[start:end,:,:]]
  if normalize: result = [normalize0to1(m) for m in result]
  if clip: result = [np.clip(255 * m, 0, 255).astype('uint8') for m in result]
  plot(result[1], "predicted image")
  return result

def evalSSIM(result):
  result1 = [np.expand_dims(m,-1) for m in result]
  raw = tf.convert_to_tensor(np.expand_dims(result1[0],0),dtype=np.float32)
  rest = tf.convert_to_tensor(np.expand_dims(result1[1],0),dtype=np.float32)
  gt = tf.convert_to_tensor(np.expand_dims(result1[2],0),dtype=np.float32)
  # Metric evaluation
  print('raw vs gt=======================')
  print("Our SSIM between raw and ground truth: ",ssim(gt,raw).numpy())
  print("RCAN paper SSIM between raw and ground truth: ",ssim_rcan(gt, raw).numpy())
  print('predicted vs gt=======================')
  print("Our SSIM between predicted and ground truth: ",ssim(gt, rest).numpy())
  print("RCAN paper SSIM between predicted and ground truth: ",ssim_rcan(gt,rest).numpy())
  print('predicted vs raw=======================')
  print("Our SSIM between predicted and raw: ",ssim(raw, rest).numpy())
  print("RCAN paper SSIM between predicted and raw: ",ssim_rcan(raw, rest).numpy())

def config(patch_size, depth):
  return {
        'img_size': 512,
        'learning_rate': 1e-4,
        'batch_size': 16,
        'alpha': 0.84,
        'patch_size':patch_size,
        'input_shape': [patch_size, patch_size],
        'kern_size':3,
        'n_depth': depth,
        'first_depth': 32,
        'dropout': 0,
        'epoch':100,
        'lr_decay_factor':0.9,
        'lr_decay_patience':5,
  }

def plot(img, label, hist=False, size=(10,10)):
  plt.figure(figsize = size)
  if hist==True:
    plt.hist(img.flatten(), bins=120)
  else: plt.imshow(img,cmap="gray")
  plt.title(label)



In [ ]:
# Run the cwtRicker, cwtMortlet

# """
# Continuous Wavelet Transforms
#     Parameters
#     ----------
#     wav:      matrix - float32 - shape (N,)
#     widthCwt: scalar - int32
#     Returns
#     -------
#     output:   matrix - float32 - shape (widthCwt, N)
# """
# def cwtRicker(tfType, wav, widthCwt):  return cwt(tfType, wav, widthCwt, rickerWavelet)
# def cwtMortlet(tfType, wav, widthCwt): return cwt(tfType, wav, widthCwt, mortletWavelet)

# # ------------------------------------------------------

# def cwt(tfType, wav, widthCwt, wavelet):
#     length = -1
#     if None not in list(tf.shape(wav)):
#       wav = tf.cast(wav, tfType)
#       wav = tf.reshape(wav, [1,length,1,1])

#     # While loop functions
#     def body(i, m):
#         v = conv1DWavelet(tfType, wav, i, wavelet)
#         v = tf.reshape(v, [length, 1])

#         m = tf.concat([m,v], 1)

#         return [1 + i, m]

#     def cond_(i, m):
#         return tf.less_equal(i, widthCwt)

#     # Initialize and run while loop
#     emptyCwtMatrix = tf.zeros([length, 0], tfType)
#     i = tf.constant(1)
#     _, result = tf.while_loop(
#             cond_,
#             body,
#             [i, emptyCwtMatrix],
#             shape_invariants=[i.get_shape(), tf.TensorShape([length, None])],
#             back_prop=False,
#             parallel_iterations=1024,
#             )
#     result = tf.transpose(result)

#     return result

# # ------------------------------------------------------
# #                 wavelets
# def rickerWavelet(tfType, scale, sampleCount):
#     def waveEquation(time):
#         time = tf.cast(time, tfType)

#         tSquare = time ** 2.
#         sigma   = 1.
#         sSquare = sigma ** 2.

#         # _1 = 2 / ((3 * a) ** .5 * np.pi ** .25)
#         _1a = (3. * sigma) ** .5
#         _1b = np.pi ** .25
#         _1 = 2. / (_1a * _1b)

#         # _2 = 1 - t**2 / a**2
#         _2 = 1. - tSquare / sSquare

#         # _3 = np.exp(-(t**2) / (2 * a ** 2))
#         _3a = -1. * tSquare
#         _3b = 2. * sSquare
#         _3 = tf.exp(_3a / _3b)

#         return _1 * _2 * _3

#     return waveletHelper(tfType, scale, sampleCount, waveEquation)

# def mortletWavelet(tfType, scale, sampleCount):
#     def waveEquation(time):
#         return tf.exp(-1. * time ** 2. / 2.) * tf.cos(5. * time) # https://www.mathworks.com/help/wavelet/ref/morlet.html

#     return waveletHelper(tfType, scale, sampleCount, waveEquation)

# def waveletHelper(tfType, scale, sampleCount, waveEquation):
#     scale         = tf.cast(scale, tfType)
#     sampleCount   = tf.cast(sampleCount, tfType)
#     unscaledTimes = tf.cast(tf.range(tf.cast(sampleCount, tf.int64)), tfType) - (sampleCount - 1.) / 2.
#     times         = unscaledTimes / scale
#     wav           = waveEquation(times)
#     wav           = wav * scale ** -.5
#     return wav

# # ------------------------------------------------------
# #                    helpers
# def conv1DWavelet(tfType, wav, waveletWidth, waveletEquation):
#     kernelSamples = waveletWidth * 10
#     kernel = waveletEquation(tfType, waveletWidth, kernelSamples)
#     kernel = tf.reverse(kernel, [0])
#     kernel = tf.reshape(kernel, tf.stack([kernelSamples,1,1,1]))

#     conv = tf.nn.convolution(wav, kernel, 'SAME')
#     conv = tf.squeeze(tf.squeeze(conv))

#     return conv

In [ ]:
def func(a):
  a = tf.reshape(4, 1)

a = tf.reshape(tf.range(0, 3)

In [ ]:
# Copied CW_SSIM (https://github.com/nickgeoca/cwt-tensorflow)
"""
Continuous Wavelet Transforms
    Parameters
    ----------
    wav:      matrix - float32 - shape (N,)
    widthCwt: scalar - int32
    Returns
    -------
    output:   matrix - float32 - shape (widthCwt, N)
"""
def cwtRicker(tfType, wav, widthCwt):  return cwt(tfType, wav, widthCwt, rickerWavelet)
def cwtMortlet(tfType, wav, widthCwt): return cwt(tfType, wav, widthCwt, mortletWavelet)

# ------------------------------------------------------

@tf.function
def cwt(tfType, wav, widthCwt, wavelet):
    print(tf.shape(wav), tf.shape(widthCwt))

    length = wav.shape[0]
    wav = tf.cast(wav, tfType)
    wav = tf.reshape(wav, tf.TensorShape([1,length,1,1]))

    # While loop functions
    def body(i, m):
        v = conv1DWavelet(tfType, wav, i, wavelet)
        v = tf.reshape(v, [length, 1])

        m = tf.concat([m,v], 1)

        return [1 + i, m]

    def cond_(i, m):
        return tf.less_equal(i, widthCwt)

    # Initialize and run while loop
    emptyCwtMatrix = tf.zeros([length, 0], tfType)
    i = tf.constant(1)
    _, result = tf.while_loop(
            cond_,
            body,
            [i, emptyCwtMatrix],
            shape_invariants=[i.get_shape(), tf.TensorShape([length, None])],
            back_prop=False,
            parallel_iterations=1024,
            )
    result = tf.transpose(result)

    return result

# ------------------------------------------------------
#                 wavelets
@tf.function
def rickerWavelet(tfType, scale, sampleCount):
    def waveEquation(time):
        time = tf.cast(time, tfType)

        tSquare = time ** 2.
        sigma   = 1.
        sSquare = sigma ** 2.

        # _1 = 2 / ((3 * a) ** .5 * np.pi ** .25)
        _1a = (3. * sigma) ** .5
        _1b = np.pi ** .25
        _1 = 2. / (_1a * _1b)

        # _2 = 1 - t**2 / a**2
        _2 = 1. - tSquare / sSquare

        # _3 = np.exp(-(t**2) / (2 * a ** 2))
        _3a = -1. * tSquare
        _3b = 2. * sSquare
        _3 = tf.exp(_3a / _3b)

        return _1 * _2 * _3

    return waveletHelper(tfType, scale, sampleCount, waveEquation)

def mortletWavelet(tfType, scale, sampleCount):
    def waveEquation(time):
        return tf.exp(-1. * time ** 2. / 2.) * tf.cos(5. * time) # https://www.mathworks.com/help/wavelet/ref/morlet.html

    return waveletHelper(tfType, scale, sampleCount, waveEquation)

@tf.function
def waveletHelper(tfType, scale, sampleCount, waveEquation):
    scale         = tf.cast(scale, tfType)
    sampleCount   = tf.cast(sampleCount, tfType)
    unscaledTimes = tf.cast(tf.range(tf.cast(sampleCount, tf.int64)), tfType) - (sampleCount - 1.) / 2.
    times         = unscaledTimes / scale
    wav           = waveEquation(times)
    wav           = wav * scale ** -.5
    return wav

# ------------------------------------------------------
#                    helpers
@tf.function
def conv1DWavelet(tfType, wav, waveletWidth, waveletEquation):
    kernelSamples = waveletWidth * 10
    kernel = waveletEquation(tfType, waveletWidth, kernelSamples)
    kernel = tf.reverse(kernel, [0])
    kernel = tf.reshape(kernel, tf.stack([kernelSamples,1,1,1]))

    conv = tf.nn.convolution(wav, kernel, padding='SAME')
    conv = tf.squeeze(tf.squeeze(conv))

    return conv

@tf.function
def cw_ssim(y_true, y_pred, width=30):
    """Compute the complex wavelet SSIM (CW-SSIM) value from the reference
    image to the target image.
    Args:
    target (str or PIL.Image): Input image to compare the reference image
    to. This may be a PIL Image object or, to save time, an SSIMImage
    object (e.g. the img member of another SSIM object).
    width: width for the wavelet convolution (default: 30)
    Returns:
    Computed CW-SSIM float value.
    """

    # print(wav.get_shape())
    # if None in wav.get_shape():
    #     # with tf.control_dependencies(checks):
    #     wav = tf.identity(tf.shape(wav))

    k = 0.01 #k (float): CW-SSIM configuration variable (default 0.01)
    # Define a width for the wavelet convolution
    widths = tf.range(1, width+1)

    # Use the image data as arrays
    sig1 = tf.reshape(y_pred, [-1])
    sig2 = tf.reshape(y_true, [-1])

    # Convolution

    # cwtmatr1 = signal.cwt(sig1, signal.ricker, widths)
    cwtmatr1 = cwtRicker(tf.float32, sig1, widths)
    # cwtmatr2 = signal.cwt(sig2, signal.ricker, widths)
    cwtmatr2 = cwtRicker(tf.float32, sig2, widths)

    # Compute the first term
    c1c2 = tf.math.multiply(abs(cwtmatr1), abs(cwtmatr2))
    c1_2 = tf.math.square(abs(cwtmatr1))
    c2_2 = tf.math.square(abs(cwtmatr2))
    num_ssim_1 = 2 * tf.math.reduce_sum(c1c2, axis=0) + k
    den_ssim_1 = tf.math.reduce_sum(c1_2, axis=0) + tf.math.reduce_sum(c2_2, axis=0) + k

    # Compute the second term
    c1c2_conj = tf.math.multiply(cwtmatr1, tf.math.conj(cwtmatr2))
    num_ssim_2 = 2 * tf.math.abs(tf.math.reduce_sum(c1c2_conj, axis=0)) + k
    den_ssim_2 = 2 * tf.math.reduce_sum(tf.math.abs(c1c2_conj), axis=0) + k

    # Construct the result
    ssim_map = (num_ssim_1 / den_ssim_1) * (num_ssim_2 / den_ssim_2)

    # Average the per pixel results
    index = tf.math.reduce_mean(ssim_map)

    return tf.cast(index, dtype=tf.float32)

In [ ]:
alphas = [ 0.84]
config_train = config(128, 2)
# generating data
data_gen = DataGenerator(
    config_train['input_shape'],
    config_train['batch_size'],
    transform_function=None)
[training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=config_train['patch_size'])
[training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=config_train['patch_size'])


      [PATCHIFYING COMPLETED] output shape, slide:  (10486, 128, 128) 64 ; number of images:  214 , number of patches:  10486
      [PATCHIFYING COMPLETED] output shape, slide:  (1666, 128, 128) 64 ; number of images:  34 , number of patches:  1666
      [PATCHIFYING COMPLETED] output shape, slide:  (10486, 128, 128) 64 ; number of images:  214 , number of patches:  10486
      [PATCHIFYING COMPLETED] output shape, slide:  (1666, 128, 128) 64 ; number of images:  34 , number of patches:  1666


In [ ]:
img1, img2 = [np.expand_dims(training_data[0],-1),], [np.expand_dims(training_data_labels[0],-1),]
assert tf.reduce_all(tf.equal(tf.shape(img1), tf.shape(img2)))

cw_ssim(tf.cast(img1,dtype=tf.float32), tf.cast(img2,dtype=tf.float32))

Tensor("Shape:0", shape=(1,), dtype=int32) Tensor("Shape_1:0", shape=(1,), dtype=int32)


InvalidArgumentError: ignored

In [ ]:
### Experiment B: testing wavelet SSIM

alphas = [ 0.84]
config_train = config(128, 2)
# generating data
data_gen = DataGenerator(
    config_train['input_shape'],
    config_train['batch_size'],
    transform_function=None)
[training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=config_train['patch_size'])
[training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=config_train['patch_size'])

tdata = data_gen.flow(*list(zip([training_data[:1000],training_data_labels[:1000]])))
vdata = data_gen.flow(*list(zip([val_data[:100],val_data_labels[:100]])))

if True:
  res = np.zeros((1,3))
  for alpha in alphas:
    curr_loss = genSSIML1_loss(alpha = alpha)
    print("EXPERIMENT 2: ALPHA - Current Alpha: ", alpha)
    # tf.config.run_functions_eagerly(True) # to enable tensor to numpy

    # learning rate reducer callback
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=config_train['lr_decay_factor'],min_delta=0,patience=config_train['lr_decay_patience'],)

    # TRAINING
    model = build_model(kern_size=config_train['kern_size'], n_depth=config_train['n_depth'],shape=(config_train['input_shape'][0],config_train['input_shape'][0],1))
    model = compile_model(model,config_train['learning_rate'], loss=curr_loss)
    print("    [TRAINING] - Current Alpha: ", alpha)
    history = model.fit(
                        x=tdata,
                        epochs = config_train['epoch'],
                        validation_data = vdata
                        , callbacks=[reduce_lr,tb_callback])
    print("    [TRAINING FINISHED] "+"a: "+str(alpha)+ ", results: val_ssim = ",np.amax(history.history['val_ssim'])," ssim = ",np.amax(history.history['ssim']))
    save_path = MAIN_PATH+"/"+MODEL_ROOT_NAME+"/"+"a_valssim_"+str(alpha)+ "_"+str(np.amax(history.history['val_ssim']))
    print("    [SAVING MODEL] path is: ",save_path)
    model.save(save_path)
    display_training_stats(history,save_path, 'loss', 'Epoch')
    display_training_stats(history,save_path, 'ssim', 'Epoch')
    display_training_stats(history,save_path, 'psnr', 'Epoch')

    res = np.concatenate((res, [[alpha,np.amax(history.history['ssim']),np.amax(history.history['val_ssim'])]]), axis=0)
    print("[CURRENT RESULT: ]")
    for i in res:
      print(i)

    # evaluating SSIM on merged images
    result = mergeAndPredict(model,val_data,val_data_labels,0, config_train['img_size'], config_train['patch_size'], config_train['patch_size']/2)
    evalSSIM(result)
    print("END TRIAL =========================================")
    print("")
    tf.keras.backend.clear_session()
    del model

  print("[TUNING  FINISHED, PRINTING RESULTS]")
  for i in res:
    print(i)

      [PATCHIFYING COMPLETED] output shape, slide:  (10486, 128, 128) 64 ; number of images:  214 , number of patches:  10486
      [PATCHIFYING COMPLETED] output shape, slide:  (1666, 128, 128) 64 ; number of images:  34 , number of patches:  1666
      [PATCHIFYING COMPLETED] output shape, slide:  (10486, 128, 128) 64 ; number of images:  214 , number of patches:  10486
      [PATCHIFYING COMPLETED] output shape, slide:  (1666, 128, 128) 64 ; number of images:  34 , number of patches:  1666
1000
100
EXPERIMENT 2: ALPHA - Current Alpha:  0.84
CUSTOM UNET INPUT SHAPE:  (128, 128, 1)
n_dim 2
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 128, 128, 1  0           []                               
                                )]                                                     

ValueError: ignored

In [ ]:
# Experiment 1: Evaluate SSIML1, SSIMVar, SSIMVarL1 losses
# The goal of this experiment is to determine whether SSIMVarL1 outperforms other Losses.

# **Methods**: run a script which runs a model over 3 losses with same configuration.

# - Epoch: 20, learning rate: 5e-5, batch size: 64, patch size: 128, overlap: half, depth: 2
# - Perform SSIM evaluation on merged images at the end.

losses = {"genSSIML1_loss": genSSIML1_loss,
          "genSSIMVar_loss": genSSIMVar_loss,
          "genSSIMVarL1_loss": genSSIMVarL1_loss,
          }

if False:
  res = np.zeros((1,3))
  for genLoss in losses.keys():
    curr_loss = losses[genLoss](alpha = 0.84)
    print("EXPERIMENT 1: LOSSES - Current Loss: ", genLoss)
    config_train = config(128, 2)

    # learning rate reducer callback
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=config_train['lr_decay_factor'],min_delta=0,patience=config_train['lr_decay_patience'],)

    # generating data
    data_gen = DataGenerator(
        config_train['input_shape'],
        config_train['batch_size'],
        transform_function=None)
    [training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=config_train['patch_size'])
    [training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=config_train['patch_size'])
    tdata = data_gen.flow(*list(zip([training_data,training_data_labels])))
    vdata = data_gen.flow(*list(zip([val_data,val_data_labels])))
    # TRAINING
    model = build_model(kern_size=config_train['kern_size'], n_depth=config_train['n_depth'],shape=(config_train['input_shape'][0],config_train['input_shape'][0],1))
    model = compile_model(model,config_train['learning_rate'], loss=curr_loss)
    print("    [TRAINING] - Current Loss: ", genLoss)
    history = model.fit(
                        x=tdata,
                        epochs = config_train['epoch'],
                        validation_data = vdata
                        , callbacks=[reduce_lr,tb_callback])
    print("    [TRAINING FINISHED] "+"p: "+ genLoss + ", results: val_ssim = ",np.amax(history.history['val_ssim'])," ssim = ",np.amax(history.history['ssim']))
    save_path = MAIN_PATH+"/"+MODEL_ROOT_NAME+"/"+"p_d_valssim_"+genLoss+"_"+str(np.amax(history.history['val_ssim']))
    print("    [SAVING MODEL] path is: ",save_path)
    model.save(save_path)
    display_training_stats(history,save_path, 'loss', 'Epoch')
    display_training_stats(history,save_path, 'ssim', 'Epoch')
    display_training_stats(history,save_path, 'psnr', 'Epoch')

    res = np.concatenate((res, [[genLoss,np.amax(history.history['ssim']),np.amax(history.history['val_ssim'])]]), axis=0)
    print("[CURRENT RESULT: ]")
    for i in res:
      print(i)

    # evaluating SSIM on merged images
    result = mergeAndPredict(model,val_data,val_data_labels,0, config_train['img_size'], config_train['patch_size'], config_train['patch_size']/2)
    evalSSIM(result)
    print("END TRIAL =========================================")
    print("")
    tf.keras.backend.clear_session()
    del training_data, val_data,tdata,vdata, training_data_labels, val_data_labels, model

  print("[TUNING  FINISHED, PRINTING RESULTS]")
  for i in res:
    print(i)

## **TRAIN PLAYGROUND**
To test:
- SSIM smaller filter: 3,5
- SSIM_mov_mae loss

In [ ]:
# test train, smaller SSIM filter size, zero overlap
config_train = config(64, 4)
config_train['learning_rate'] = 1e-4
# learning rate reducer callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=config_train['lr_decay_factor'],min_delta=0,patience=config_train['lr_decay_patience'],)

# generating data
data_gen = DataGenerator(
    config_train['input_shape'],
    config_train['batch_size'],
    transform_function=None)
[training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=config_train['patch_size'])
[training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=config_train['patch_size'])
tdata = data_gen.flow(*list(zip([training_data,training_data_labels])))
vdata = data_gen.flow(*list(zip([val_data,val_data_labels])))
# TRAINING
model = build_model(kern_size=config_train['kern_size'], n_depth=config_train['n_depth'],shape=(config_train['input_shape'][0],config_train['input_shape'][0],1))
model = compile_model(model,config_train['learning_rate'])
print("    [TRAINING] "+"patch size: "+ str(config_train['patch_size']) + ", current depth: "+str(config_train['n_depth']))
history = model.fit(
                    x=tdata,
                    epochs = config_train['epoch'],
                    validation_data = vdata
                    , callbacks=[reduce_lr,tb_callback])
print("    [TRAINING FINISHED] "+"p: "+ str(config_train['patch_size']) + ", d: "+str(config_train['n_depth'])+ ", results: val_ssim = ",np.amax(history.history['val_ssim'])," ssim = ",np.amax(history.history['ssim']))
save_path = MAIN_PATH+"/"+MODEL_ROOT_NAME+"/"+"p_d_valssim_"+str(config_train['patch_size'])+"_"+str(config_train['n_depth'])+"_"+str(np.amax(history.history['val_ssim']))
print("    [SAVING MODEL] path is: ",save_path)
model.save(save_path)
display_training_stats(history,save_path, 'loss', 'Epoch')
display_training_stats(history,save_path, 'ssim', 'Epoch')
display_training_stats(history,save_path, 'psnr', 'Epoch')

In [ ]:
test_loss = genSSIMVar_loss(alpha=0.3)
# test new loss, no overlap, ssim filter 3
config_train = config(64, 3)
config_train['learning_rate'] = 1e-4
# learning rate reducer callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=True,factor=config_train['lr_decay_factor'],min_delta=0,patience=config_train['lr_decay_patience'],)

# generating data
data_gen = DataGenerator(
    config_train['input_shape'],
    config_train['batch_size'],
    transform_function=None)
[training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=config_train['patch_size'])
[training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=config_train['patch_size'])
tdata = data_gen.flow(*list(zip([training_data,training_data_labels])))
vdata = data_gen.flow(*list(zip([val_data,val_data_labels])))
# TRAINING
model = build_model(kern_size=config_train['kern_size'], n_depth=config_train['n_depth'],shape=(config_train['input_shape'][0],config_train['input_shape'][0],1))
model = compile_model(model,config_train['learning_rate'], loss=test_loss)
print("    [TRAINING] "+"patch size: "+ str(config_train['patch_size']) + ", current depth: "+str(config_train['n_depth']))
history = model.fit(
                    x=tdata,
                    epochs = config_train['epoch'],
                    validation_data = vdata
                    , callbacks=[reduce_lr,tb_callback])
print("    [TRAINING FINISHED] "+"p: "+ str(config_train['patch_size']) + ", d: "+str(config_train['n_depth'])+ ", results: val_ssim = ",np.amax(history.history['val_ssim'])," ssim = ",np.amax(history.history['ssim']))
save_path = MAIN_PATH+"/"+MODEL_ROOT_NAME+"/"+"p_d_valssim_"+str(config_train['patch_size'])+"_"+str(config_train['n_depth'])+"_"+str(np.amax(history.history['val_ssim']))
print("    [SAVING MODEL] path is: ",save_path)
model.save(save_path)
display_training_stats(history,save_path, 'loss', 'Epoch')
display_training_stats(history,save_path, 'ssim', 'Epoch')
display_training_stats(history,save_path, 'psnr', 'Epoch')

In [ ]:
tf.keras.backend.clear_session()


In [ ]:
# [training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, patch=None)
# [training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH,patch=None)


In [ ]:
# # print(np.amax(val_data[0]))
# plot(val_data[0],'test',hist=True)
# [training_dataa, val_dataa] = load_data(MAIN_PATH+DATA_PATH, patch=32)
# print("TEST AFTER", np.amax(training_dataa))
# test = normalize(merge_patches(val_dataa[0:961,:,:],512,32,16))
# plot(test,'test',hist=True)


In [ ]:
# result = predictAndPlot(model,val_data,val_data_labels, 0, 961, isPlot=True, merge=True, normalize=False, clip=False)
# evalSSIM(result)

In [ ]:
def revert_img(img,original_size, patch_shape, slide):
  # reverts original image and removes overlaps by splitting overlap over 2 images
  step = int(patch_shape-slide)
  reconstructed_arr = np.zeros((original_size,original_size))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      start_x = int(slide/2)
      start_y = int(slide/2)
      end_x = 0
      end_y = 0
      if x == 0:
        start_x = 0
        end_x = int(slide/2)
      if y == 0:
        start_y = 0
        end_y = int(slide/2)
      if x == img.shape[0]-1: end_x = int(slide/2)
      if y == img.shape[1]-1: end_y = int(slide/2)
      x_pos, y_pos = x * step + start_x, y * step + start_y
      # print('x_pos, start_x: ',x_pos, start_x)
      # print('x_pos + step + end_x, start_x+step+end_x: ',x_pos + step + end_x, start_x+step+end_x)
      # print('y_pos, start_y: ',y_pos, start_y)
      # print('y_pos + step + end_y,  start_y+step+end_y: ',y_pos + step + end_y,  start_y+step+end_y)
      # print("end===============x,y: ",x,y)
      reconstructed_arr[x_pos : x_pos + step + end_x, y_pos : y_pos + step + end_y] = img[x, y, start_x:start_x+step+end_x, start_y:start_y+step+end_y]
  return reconstructed_arr

def merge_patches(img, original_size, patch_shape, slide):
  #  merging patches, img is a 3D array of stacked patches
  print("merging patches, img shape: ", img.shape)
  row_len = int(math.sqrt(img.shape[0]))
  patches = np.zeros((row_len,row_len,patch_shape,patch_shape))
  print(img.shape)
  print(patches.shape)
  for r in range(row_len):
      patches[r,:,:,:] = img[r*row_len:r*row_len+row_len,:,:]
  # plt.figure(5)
  # plt.imshow(patches[1,1,:,:])
  return revert_img(patches,original_size,patch_shape, slide)


def MSE(img1, img2):
  # comparing one processed and preprocessed image
  squared_diff = (img1 -img2) ** 2
  summed = np.sum(squared_diff)
  num_pix = img1.shape[0] * img1.shape[1] #img1 and 2 should have same shape
  err = summed / num_pix
  return err

def normalize(img):
  # Normalizing images between 0 and 1 and preserving distribution
  img_norm = (img - np.amin(img))/( np.amax(img)- np.amin(img))
  return img_norm

def plot(img, label, hist=False, size=(10,10)):
  plt.figure(figsize = size)
  if hist==True:
    plt.hist(img.flatten(), bins=120)
  else: plt.imshow(img,cmap="gray")
  plt.title(label)

def normalize_between_zero_and_one(m):
    max_val, min_val = m.max(), m.min()
    diff = max_val - min_val
    return (m - min_val) / diff if diff > 0 else np.zeros_like(m)

def normalize_percentile(img, pmin=0.1, pmax=99.9, clip = True):
  eps=1e-20 # avoid zero division
  mi = np.percentile(img,pmin,axis=None,keepdims=True)
  # mi = np.amin(img)
  # print("mi",mi)
  ma = np.percentile(img,pmax,axis=None,keepdims=True)
  if clip == True: return np.clip((img - mi) / ( ma - mi + eps ), 0, 1)
  return (img - mi) / ( ma - mi + eps )

# def ssim(y_true, y_pred):
#     '''
#     Computes the structural similarity index between two images. Note that the
#     maximum signal value is assumed to be 1.
#     '''

#     return ssim2(y_true, y_pred,1,k2=0.05)
# our SSIM loss
from tensorflow.image import ssim as ssim2
def ssim_our(y_true, y_pred):
  return ssim2(y_true, y_pred,1,k2=0.05)

def _get_gaussian_kernel(dim, size, sigma):
    k = size // 2
    normal = tfp.distributions.Normal(0.0, sigma)
    p = normal.prob(tf.range(-k, size - k, dtype=tf.float32))

    indices = [chr(i) for i in range(105, 105 + dim)]
    eq = ','.join(indices) + '->' + ''.join(indices)
    kernel = tf.einsum(eq, *([p] * dim))
    kernel /= tf.reduce_sum(kernel)
    kernel = kernel[..., tf.newaxis, tf.newaxis]

    return kernel

# RCAN ssim
import keras.backend as K
def ssim_rcan(y_true, y_pred):
    '''
    Computes the structural similarity index between two images. Note that the
    maximum signal value is assumed to be 1.
    References
    ----------
    Image Quality Assessment: From Error Visibility to Structural Similarity
    https://doi.org/10.1109/TIP.2003.819861
    '''

    c1 = 0.01 ** 2
    c2 = 0.03 ** 2

    dim = K.ndim(y_pred) - 2
    if dim not in (2, 3):
        raise NotImplementedError(f'{dim}D SSIM is not suported')

    num_channels = K.int_shape(y_pred)[-1]

    kernel = _get_gaussian_kernel(dim, 11, 1.5)
    conv = K.conv2d if dim == 2 else K.conv3d

    def average(x):
        # channel-wise weighted average using the Gaussian kernel
        return tf.concat(
            [conv(y, kernel) for y in tf.split(x, num_channels, axis=-1)],
            axis=-1)

    ux = average(y_true)
    uy = average(y_pred)

    a = ux * uy
    b = K.square(ux) + K.square(uy)
    c = average(y_true * y_pred)
    d = average(K.square(y_true) + K.square(y_pred))

    lum = (2 * a + c1) / (b + c1)
    cs = (2 * (c - a) + c2) / (d - b + c2)

    return K.mean(K.batch_flatten(lum * cs), axis=-1)

def predictAndPlot(model,input,val, start, end, isPlot=True, merge=True, normalize=False, clip=False):
  pred = model.predict(input[start:end])
  print("shape of predicted image: ", np.shape(pred))
  if len(np.shape(input)) > 3:
    if merge:
      result = [merge_patches(input[start:end,:,:,0],512,64,0), merge_patches(pred[start:end,:,:,0],512,64,0), merge_patches(val[start:end,:,:,0],512,64,0)]
    else: result = [input[start:end,:,:,0], pred[start:end,:,:,0],val[start:end,:,:,0]]
  else:
    if merge:
      result = [merge_patches(input[start:end,:,:],512,64,0), merge_patches(pred[start:end,:,:,0],512,64,0), merge_patches(val[start:end,:,:],512,64,0)]
    else: result = [input[start:end,:,:], pred[start:end,:,:,0], val[start:end,:,:]]
  if normalize: result = [normalize_percentile(m) for m in result]
  if clip: result = [np.clip(255 * m, 0, 255).astype('uint8') for m in result]
  res_img =  np.concatenate((np.concatenate((result[0],result[1]),axis=1),result[2]),axis=1)
  plot(res_img, "input, restored, gt", size=(30,30))
  # plot(result[0],'input 25x')
  # plot(result[1],'restored')
  # plot(result[2],'gt 40x')
  if isPlot:
    plot(result[0],'hist input 25x',hist=True)
    plot(result[1],'hist restored',hist=True)
    plot(result[2],'hist gt 40x',hist=True)
  return result

def evalSSIM(result):
  result1 = [np.expand_dims(m,-1) for m in result]
  raw = tf.convert_to_tensor(np.expand_dims(result1[0],0),dtype=np.float32)
  rest = tf.convert_to_tensor(np.expand_dims(result1[1],0),dtype=np.float32)
  gt = tf.convert_to_tensor(np.expand_dims(result1[2],0),dtype=np.float32)
  # Metric evaluation
  print('raw vs gt=======================')
  print("Our SSIM between raw and ground truth: ",ssim_our(gt,raw).numpy())
  print("RCAN paper SSIM between raw and ground truth: ",ssim_rcan(gt, raw).numpy())
  print('predicted vs gt=======================')
  print("Our SSIM between predicted and ground truth: ",ssim_our(gt, rest).numpy())
  print("RCAN paper SSIM between predicted and ground truth: ",ssim_rcan(gt,rest).numpy())
  print('predicted vs raw=======================')
  print("Our SSIM between predicted and raw: ",ssim_our(raw, rest).numpy())
  print("RCAN paper SSIM between predicted and raw: ",ssim_rcan(raw, rest).numpy())

In [ ]:
import tensorflow_probability as tfp
result = predictAndPlot(model,expandLastDim(val_data), expandLastDim(val_data_labels), 0, 64, isPlot=True, merge=True, normalize=False, clip=False)
evalSSIM(result)

In [ ]:
print(np.shape(val_data))